Financial audit 


In [6]:
import plotly
import matplotlib as mpl
from matplotlib import pyplot as plt

import pandas as pd
import plotly.graph_objects as go
import numpy as np


In [7]:
data = pd.read_csv("C:/Users/bruno/OneDrive/Admin/Misc/FINANCIAL_AUDIT.csv")

#replace all NA 
data = data.fillna("notspecified")

#clean data replace all " " with "" even within strings
data = data.replace(" ", "", regex=True)

# #select elements after jun 1st 2024
# data['n1'] = pd.to_datetime(data['n1'],format="%d/%m/%Y")
# data = data[data['n1'] >= '2024-06-01']

data

,n1,n2,n3,n4,n5,Total,Amount,Cat1,Cat2,Unnamed: 9
0,07/01/2024,2024010701,EFTPOS,Toni'sPizzeriaAuckland,EFTPOS,441.00,-33.00,food,restaurant,tonis
1,07/01/2024,2024010702,EFTPOS,Toni'sPizzeriaAuckland,EFTPOS,439.00,-2.00,food,restaurant,tonis
2,07/01/2024,2024010703,D/C,D/CFROMFromMISSJIYUNPARK,xmas,539.00,100.00,jennifer,notspecified,notspecified
3,07/01/2024,2024010704,EFTPOS,ZMtAlbertAuckland,EFTPOS,481.29,-57.71,transport,fuel,notspecified
4,08/01/2024,2024010801,D/C,D/CFROMFromMISSJIYUNPARK,rentdonation,1981.29,1500.00,jennifer,notspecified,notspecified
...,...,...,...,...,...,...,...,...,...,...
667,16/11/2024,2024111602,D/C,D/CFROMFromMISSJIYUNPARK,rent,9207.48,918.53,jennifer,notspecified,notspecified
668,16/11/2024,2024111603,DEBIT,DEBIT,IMTUSD1152.20MCRMCRMCR,7202.48,-2005.00,activities,mcr,notspecified
669,16/11/2024,2024111604,EFTPOS,TEND.NZAUCKLAND,EFTPOS,7173.48,-29.00,misc,doctor,notspecified
670,16/11/2024,2024111605,EFTPOS,UBER*TRIPCHRISTCHURCH,EFTPOS,7157.05,-16.43,transport,uber,notspecified


In [8]:
# Separate income and expenses and generate the different levels
income = data[data['Amount'] > 0]
expenses = data[data['Amount'] < 0]

# income first level
income_l1 = income.groupby(['Cat1', 'Cat2'])['Amount'].sum().reset_index()
income_l1.rename(columns={'Cat1': 'target', 'Cat2': 'source'}, inplace=True)
print(income_l1)

# income second level
income_l2 = income.groupby(['Cat1'])['Amount'].sum().reset_index()
income_l2.rename(columns={'Cat1': 'source'}, inplace=True)
income_l2['target'] = 'Total Income'
print(income_l2)

#add a "_" to the cat1 and cat2 columns of expenses
expenses.loc[:,'Cat1'] = expenses.loc[:,'Cat1'] + "_"
expenses.loc[:,'Cat2'] = expenses.loc[:,'Cat2'] + "_"

# expenses first level
expenses_l1 = expenses.groupby(['Cat1'])['Amount'].sum().reset_index()
expenses_l1.rename(columns={'Cat1': 'target'}, inplace=True)
expenses_l1['source'] = 'Total Income'
print(expenses_l1)

# expenses second level
expenses_l2 = expenses.groupby(['Cat1', 'Cat2'])['Amount'].sum().reset_index()
expenses_l2.rename(columns={'Cat1': 'source', 'Cat2': 'target'}, inplace=True)
print(expenses_l2)


        target             source    Amount
0          IRD             rebate   2524.33
1         bond       notspecified   1750.00
2     internal       notspecified    109.32
3     jennifer               debt   3000.00
4     jennifer           donation    700.00
5     jennifer       notspecified   2518.53
6     jennifer           usa_rent    530.00
7         misc        pocketmoney   1200.00
8         rent           jennfier    300.00
9         rent      jennifer_rent   5100.00
10        rent       notspecified    300.00
11  rentrefund       notspecified    600.00
12      salary       notspecified  49466.27
13      salary      reimbursement   1632.24
14    shopping       notspecified    140.00
15      travel  jennifer_transfer    900.00
16      travel       notspecified   2060.01
       source    Amount        target
0         IRD   2524.33  Total Income
1        bond   1750.00  Total Income
2    internal    109.32  Total Income
3    jennifer   6748.53  Total Income
4        misc   12

In [9]:
# Prepare the nodes and links
nodes = []
links = []

# add nodes
nodes.extend(income['Cat1'].unique())
nodes.extend(income['Cat2'].unique())
nodes.extend(expenses['Cat1'].unique())
nodes.extend(expenses['Cat2'].unique())
nodes.append("Total Income")

print(nodes)

# Create income l1 links
for _, row in income_l1.iterrows():
    links.append({
        'source': nodes.index(row['source']),
        'target': nodes.index(row['target']),
        'value': abs(row['Amount'])
    })

# create income l2 linkes
for _, row in income_l2.iterrows():
    links.append({
        'source': nodes.index(row['source']),
        'target': nodes.index(row['target']),
        'value': abs(row['Amount'])
    })

# Create links from Total Income to expense_l1
for _, row in expenses_l1.iterrows():
    links.append({
        'source': nodes.index(row["source"]),
        'target': nodes.index(row['target']),
        'value': abs(row['Amount'])
    })

#Create links from expense_l1 to expense_l2
for _, row in expenses_l2.iterrows():
    links.append({
        'source': nodes.index(row["source"]),
        'target': nodes.index(row['target']),
        'value': abs(row['Amount'])
    })

# Create the Sankey diagram
fig = go.Figure(data=[go.Sankey(
    node = dict(
        pad = 15,
        thickness = 20,
        line = dict(color = "black", width = 0.5),
        label = nodes,
        color = "blue"
    ),
    link = dict(
        source = [link['source'] for link in links],
        target = [link['target'] for link in links],
        value = [link['value'] for link in links],
        color = [f'rgba({np.random.randint(0,256)},{np.random.randint(0,256)},{np.random.randint(0,256)},0.5)' for _ in links]
    )
)])

# Update the layout
fig.update_layout(
    title_text="Income and Expense Flow", 
    font_size=10,
    height=800  # Make the diagram taller
)

# Show the figure
fig.show()

['jennifer', 'misc', 'rent', 'shopping', 'salary', 'travel', 'internal', 'IRD', 'rentrefund', 'bond', 'notspecified', 'pocketmoney', 'jennifer_rent', 'reimbursement', 'debt', 'jennifer_transfer', 'donation', 'usa_rent', 'jennfier', 'rebate', 'food_', 'transport_', 'misc_', 'rent_', 'activities_', 'shopping_', 'jennifer_', 'travel_', 'usarent_', 'monkey_', 'admin_', 'restaurant_', 'fuel_', 'gym_', 'fastfood_', 'groceries_', 'notspecified_', 'parking_', 'utilities_', 'AT_', 'misc_', 'chatgpt_', 'tax_', 'treat_', 'date_', 'gift_', 'monitor_', 'chemist_', 'nl_', 'debt_', 'farmers_', 'unknown_', 'mecca_', 'game_', 'warehouse_', 'at_', 'jmart_', 'paganini_', 'printing_', 'dentist_', 'plane_', 'insurance_', 'whitcoulls_', 'massage_', 'ubereats_', '?_', 'uber_', 'haircut_', 'bill_', 'goshuttle_', 'train_', 'transfer_', 'rego_', 'bowl_', 'laundry_', 'kmart_', 'foodtransfer_', 'beam_', 'dress_', 'tickets_nov_', 'steam_', 'registration_', 'dyson_', 'flowers_', 'csanz_', 'RDA_', 'dresss_', 'wof_',

In [90]:
cat1_nodes = expenses['Cat1'].unique()
cat1_nodes

array(['shopping_', 'jennifer_', 'misc_', 'transport_', 'monkey_',
       'food_', 'travel_', 'activities_', 'admin_'], dtype=object)